<a href="https://colab.research.google.com/github/VSCHY/RiverNetwork/blob/master/Flooded_area_Pantanal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Flooded area over Pantanal</h1>
<h2>Installation</h2>


In [1]:
!pip install -U -q earthengine-api
!pip install -U -q folium

     |████████████████████████████████| 163kB 2.9MB/s 
     |████████████████████████████████| 102kB 2.3MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.11.0 which is incompatible.


In [2]:
!pip install -U -q geehydro

In [3]:
!earthengine authenticate

Instructions for updating:
non-resource variables are not supported in the long term
Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=xfCJPrbQpjXCxcZioq1hbQSR61IWvIZtPZQcNCyWqHE&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/4wG_UMd3JItkK3tfi5zcLNnbr9hVE9Q0bf7KJqvk8f1E_pnvwCjf9XY

Successfully saved authorization token.


<h2>Initialization</h2>

In [4]:
import ee
import folium
import geehydro

try:
  ee.Initialize()
  print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
  print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


<h2> First approach : explore MODIS DATA </h2>
From MODIS dataset 2 products are commonly used for flood detection : MOD09A1/A4 and MOD43A4.

*  MOD09A1.006 Terra Surface Reflectance 8-Day Global 500m
*  MCD43A4.006 MODIS Nadir BRDF-Adjusted Reflectance Daily 500m

In this case I explore MCD43A4. I compare a "wet period" (April 2019) to a "dry period" (August 2019) for the Natural Visible colors and for the short wave infrared colors.

In [5]:
poi = ee.Geometry.Point([-57, -18.8])
MOD = ee.ImageCollection("MODIS/006/MCD43A4").filterBounds(poi)

MOD_dry = MOD.filterDate('2019-08-01', '2019-08-30').min()
MOD_dry_rgb = ee.Image(MOD_dry).visualize(min=0, max=2000, bands=['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band3'])
MOD_dry_swir = ee.Image(MOD_dry).visualize(min=50, max=3300, bands=['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6'])

MOD_wet = MOD.filterDate('2019-04-01', '2020-04-30').min()
MOD_wet_rgb = ee.Image(MOD_wet).visualize(min=0, max=2000, bands=['Nadir_Reflectance_Band1', 'Nadir_Reflectance_Band4', 'Nadir_Reflectance_Band3'])
MOD_wet_swir = ee.Image(MOD_wet).visualize(min=50, max=3300, bands=['Nadir_Reflectance_Band2', 'Nadir_Reflectance_Band5', 'Nadir_Reflectance_Band6'])


myMap = folium.Map(location=[-18.8, -57], zoom_start=8, tiles='Stamen Terrain', control_scale=False, detect_retina=True,width=600, height=600)
myMap.setOptions('HYBRID')

myMap.addLayer(MOD_dry_rgb, {'overlay':True}, 'Natural Colour DRY')
myMap.addLayer(MOD_dry_swir, {'overlay':True}, 'Short-wave Infrared DRY')
myMap.addLayer(MOD_wet_rgb, {'overlay':True}, 'Natural Colour WET')
myMap.addLayer(MOD_wet_swir, {'overlay':True}, 'Short-wave Infrared WET')

# Add a layer control widget to the map
folium.LayerControl().add_to(myMap)

# Show the map!
display(myMap)


<h3>Flood indexes</h3>
There is a large range of flood detection related indexes. Some of the most basic are presented below. They can be applied to other product even if the description of the bands may slightly change.

*   Normalized Difference Water Index : water has positive values 

$NDWI = \frac{(NIR-SWIR-2)}{(NIR+SWIR-2)}$

*   Modified Normalized Difference Water Index : water has positive values

$mNDWI = \frac{(Green-MIR)}{(Green+MIR)}$

* Water Ratio Index : 

$WRI = \frac{(Green+Red)}{(NIR+SWIR)}$

For MCD43A4 : Red is band 1, NIR is band 2, Green is band 4 and MIR is band 7

In [18]:
poi = ee.Geometry.Point([-57, -18.8])
MOD = ee.ImageCollection("MODIS/006/MCD43A4").filterBounds(poi)

# Green is band 4 and MIR band 6
green = 'Nadir_Reflectance_Band4'
MIR = 'Nadir_Reflectance_Band7'
NIR = 'Nadir_Reflectance_Band2'
SWIR= 'Nadir_Reflectance_Band6'
red = 'Nadir_Reflectance_Band1'
minmNDWI= -1; maxmNDWI = 0.5
minNDWI= 0; maxNDWI = 0.5
minWRI = 50; maxWRI = 200

# DRY DATE
MOD_dry = MOD.filterDate('2019-08-01', '2019-08-30').min()
MOD_DRY_NDWI = MOD_dry.select(NIR).subtract(MOD_dry.select(SWIR)).divide(MOD_dry.select(NIR).add(MOD_dry.select(SWIR)))
MOD_DRY_NDWI = MOD_DRY_NDWI.visualize(min=minNDWI, max=maxNDWI)
#
MOD_DRY_mNDWI = MOD_dry.select(green).subtract(MOD_dry.select(MIR)).divide(MOD_dry.select(green).add(MOD_dry.select(MIR)))
MOD_DRY_mNDWI = MOD_DRY_mNDWI.visualize(min=minmNDWI, max=maxmNDWI)
#
WRI_DRY = MOD_dry.select(green).add(MOD_dry.select(red)).divide(MOD_dry.select(NIR).add(MOD_dry.select(SWIR)))
WRI_DRY = MOD_DRY_NDWI.visualize(min=minWRI, max=maxWRI)
# WET DATE

MOD_wet = MOD.filterDate('2019-04-01', '2019-04-30').min()
MOD_WET_NDWI = MOD_wet.select(NIR).subtract(MOD_wet.select(SWIR)).divide(MOD_wet.select(NIR).add(MOD_wet.select(SWIR)))
MOD_WET_NDWI = MOD_WET_NDWI.visualize(min=minNDWI, max=maxNDWI)
#
MOD_WET_mNDWI = MOD_wet.select(green).subtract(MOD_wet.select(MIR)).divide(MOD_wet.select(green).add(MOD_wet.select(MIR)))
MOD_WET_mNDWI = MOD_WET_mNDWI.visualize(min=minmNDWI, max=maxmNDWI)
#
WRI_WET = MOD_wet.select(green).add(MOD_wet.select(red)).divide(MOD_wet.select(NIR).add(MOD_wet.select(SWIR)))
WRI_WET = MOD_WET_NDWI.visualize(min=minWRI, max=maxWRI)

myMap = folium.Map(location=[-18.8, -57], zoom_start=8, tiles='Stamen Terrain', control_scale=True, detect_retina=True,width=600, height=600)
myMap.setOptions('HYBRID')

myMap.addLayer(MOD_DRY_mNDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'mNDWI DRY')
myMap.addLayer(MOD_WET_mNDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'mNDWI WET')
myMap.addLayer(MOD_DRY_NDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'NDWI DRY')
myMap.addLayer(MOD_WET_NDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'NDWI WET')
myMap.addLayer(WRI_DRY, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'WRI DRY')
myMap.addLayer(WRI_WET, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'WRI WET')
#myMap.addLayer(MOD_dry_swir, {'overlay':True}, 'Short-wave Infrared DRY')
#myMap.addLayer(MOD_wet_rgb, {'overlay':True}, 'Natural Colour WET')
#myMap.addLayer(MOD_wet_swir, {'overlay':True}, 'Short-wave Infrared WET')

# Add a layer control widget to the map
folium.LayerControl().add_to(myMap)

# Show the map!
display(myMap)

<h2>Landsat</h2>
The previous 

In [27]:
poi = ee.Geometry.Point([-57, -18.8])
MOD = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(poi)

MOD_dry = MOD.filterDate('2019-07-01', '2020-07-30').min()
# Green is band 4 and MIR band 6
green = 'B3'
MIR = 'B7'
NIR = 'B5'
red = 'B1'
SWIR = 'B7'
minmNDWI= -1; maxmNDWI = 0.4
minNDWI= 0; maxNDWI = 0.9
minWRI = 50; maxWRI = 210

# DRY DATE
MOD_dry = MOD.filterDate('2019-07-15', '2019-08-31').min()
MOD_DRY_NDWI = MOD_dry.select(NIR).subtract(MOD_dry.select(SWIR)).divide(MOD_dry.select(NIR).add(MOD_dry.select(SWIR)))
MOD_DRY_NDWI = MOD_DRY_NDWI.visualize(min=minNDWI, max=maxNDWI)
#
MOD_DRY_mNDWI = MOD_dry.select(green).subtract(MOD_dry.select(MIR)).divide(MOD_dry.select(green).add(MOD_dry.select(MIR)))
MOD_DRY_mNDWI = MOD_DRY_mNDWI.visualize(min=minmNDWI, max=maxmNDWI)
#
WRI_DRY = MOD_dry.select(green).add(MOD_dry.select(red)).divide(MOD_dry.select(NIR).add(MOD_dry.select(SWIR)))
WRI_DRY = MOD_DRY_NDWI.visualize(min=minWRI, max=maxWRI)
# WET DATE

MOD_wet = MOD.filterDate('2019-04-01', '2019-04-30').min()
MOD_WET_NDWI = MOD_wet.select(NIR).subtract(MOD_wet.select(SWIR)).divide(MOD_wet.select(NIR).add(MOD_wet.select(SWIR)))
MOD_WET_NDWI = MOD_WET_NDWI.visualize(min=minNDWI, max=maxNDWI)
#
MOD_WET_mNDWI = MOD_wet.select(green).subtract(MOD_wet.select(MIR)).divide(MOD_wet.select(green).add(MOD_wet.select(MIR)))
MOD_WET_mNDWI = MOD_WET_mNDWI.visualize(min=minmNDWI, max=maxmNDWI)
#
WRI_WET = MOD_wet.select(green).add(MOD_wet.select(red)).divide(MOD_wet.select(NIR).add(MOD_wet.select(SWIR)))
WRI_WET = MOD_WET_NDWI.visualize(min=minWRI, max=maxWRI)

myMap = folium.Map(location=[-18.8, -57], zoom_start=7, tiles='Stamen Terrain', control_scale=True, detect_retina=True,width=600, height=600)
myMap.setOptions('HYBRID')

myMap.addLayer(MOD_DRY_mNDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'mNDWI DRY')
myMap.addLayer(MOD_WET_mNDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'mNDWI WET')
myMap.addLayer(MOD_DRY_NDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'NDWI DRY')
myMap.addLayer(MOD_WET_NDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'NDWI WET')
myMap.addLayer(WRI_DRY, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'WRI DRY')
myMap.addLayer(WRI_WET, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'WRI WET')

myMap = folium.Map(location=[-18.8, -57], zoom_start=8, tiles='Stamen Terrain', control_scale=True, detect_retina=True,width=600, height=600)
myMap.setOptions('HYBRID')

myMap.addLayer(MOD_DRY_mNDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'mNDWI DRY')
myMap.addLayer(MOD_WET_mNDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'mNDWI WET')
myMap.addLayer(MOD_DRY_NDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'NDWI DRY')
myMap.addLayer(MOD_WET_NDWI, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'NDWI WET')
myMap.addLayer(WRI_DRY, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'WRI DRY')
myMap.addLayer(WRI_WET, {'overlay':True, "palette": ['blue', 'green', 'red']}, 'WRI WET')

# Add a layer control widget to the map
folium.LayerControl().add_to(myMap)

# Show the map!
display(myMap)

<h2>Tasseled Cap Wetness</h2>

The Tasseled Cap Index (Kauth and Thomas, 1976) are indicators generated from a transformation of the spectral information of satellite data. It is composed by 3 band : 

*   Brightness : for the ground
*   greeness : for the vegetation
*   Wetness : for the interaction of soil and canopy moisture

These indices are commonly used for the analysis of vegetation while the wetness band may be used for flood detection.


In [90]:
coefficients = ee.Array([
  [0.3037, 0.2793, 0.4743, 0.5585, 0.5082, 0.1863],
  [-0.2848, -0.2435, -0.5436, 0.7243, 0.0840, -0.1800],
  [0.1509, 0.1973, 0.3279, 0.3406, -0.7112, -0.4572],
  [-0.8242, 0.0849, 0.4392, -0.0580, 0.2012, -0.2768],
  [-0.3280, 0.0549, 0.1075, 0.1855, -0.4357, 0.8085],
  [0.1084, -0.9022, 0.4120, 0.0573, -0.0251, 0.0238]
]);

poi = ee.Geometry.Point([-57, -18.8])
MOD = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA').filterBounds(poi).select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7']);
MOD_wet = MOD.filterDate('2019-04-01', '2020-04-30').min()
MOD_dry = MOD.filterDate('2019-07-15', '2020-08-31').min()


# Make an Array Image, with a 1-D Array per pixel.
arrayImage1D_wet = MOD_wet.toArray();
arrayImage1D_dry = MOD_dry.toArray();

# Make an Array Image with a 2-D Array per pixel, 6x1.
arrayImage2D_wet = arrayImage1D_wet.toArray(1);
arrayImage2D_dry = arrayImage1D_dry.toArray(1);

TCW_Wet = ee.Image(coefficients).matrixMultiply(arrayImage2D_wet).arrayProject([0]).arrayFlatten(
    [['brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth']]);

TCW_Dry = ee.Image(coefficients).matrixMultiply(arrayImage2D_dry).arrayProject([0]).arrayFlatten(
    [['brightness', 'greenness', 'wetness', 'fourth', 'fifth', 'sixth']]);

vizParams_wetness = {
  "bands": ['brightness', 'greenness', 'wetness'],
  "min": 0., "max": [0.0, 0.0, 0.005]
};
vizParams_greeness = {
  "bands": ['brightness', 'greenness', 'wetness'],
  "min": -0.1, "max": [0.0, 0.001, 0.0]
};

myMap = folium.Map(location=[-18.8, -57], zoom_start=8, tiles='Stamen Terrain', control_scale=True, detect_retina=True,width=600, height=600)
myMap.setOptions('HYBRID')

#myMap.addLayer(MOD_wet, {"bands": ['B4'], "min": 0, "max":  0.1}, 'Landsat Wet');
myMap.addLayer(MOD_wet, {"bands": ['B4', 'B3', 'B2'], "min": 0, "max": [0.4, 0.4, 0.4], "gamma": [1.1, 1.2, 0.9]}, 'Landsat Wet');
myMap.addLayer(TCW_Wet, vizParams_wetness, 'TCW Wet');
myMap.addLayer(TCW_Dry, vizParams_wetness, 'TCW Dry');

# Add a layer control widget to the map
folium.LayerControl().add_to(myMap)

# Show the map!
display(myMap)
